In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
import pandas as pd
import io

from google.colab import files
uploaded = files.upload()

Saving dynamic_sensor_data_with_labels.csv to dynamic_sensor_data_with_labels.csv


In [ ]:
from io import StringIO

data = uploaded['dynamic_sensor_data_with_labels.csv'].decode('utf-8')
csv_file = StringIO(data)

In [ ]:
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

class FeatureWeightingLayer(Layer):
    def __init__(self, scaling_factors, **kwargs):
        super(FeatureWeightingLayer, self).__init__(**kwargs)
        self.scaling_factors = K.variable(scaling_factors, name='scaling_factors')

    def call(self, inputs):
        return inputs * self.scaling_factors

    def get_config(self):
        config = super(FeatureWeightingLayer, self).get_config()
        config.update({"scaling_factors": self.scaling_factors.numpy()})
        return config


In [ ]:
import pandas as pd

# Define the path to your CSV file


# Define the column names
column_names = ['angle1', 'angle2', 'angle3', 'angle4', 'angle5', 'rolldeg', 'pitchdeg', 'anglegx', 'anglegy', 'anglegz', 'label']

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file, names=column_names)
print("Initial Data:")
print(df.head())

segments = []
current_segment = {'data': [], 'label': None}
segment_counts = {}  # Dictionary to track segment counts for each label
last_label = None  # Track the last label to reset segment count

for index, row in df.iterrows():
    # Check if the row indicates the start of a new segment
    if 'Start' in str(row['angle1']):
        if current_segment['data']:  # Complete the current segment before starting a new one
            segments.append(current_segment)
            label = current_segment['label']
            if label in segment_counts:
                segment_counts[label] += 1
            else:
                segment_counts[label] = 1
        current_segment = {'data': [], 'label': str(row['angle2']).strip()}  # Reset for new segment

    elif 'End' in str(row['angle1']):
        if current_segment['data']:  # Ensure there's data to conclude
            segments.append(current_segment)
            label = current_segment['label']
            if label not in segment_counts:
                segment_counts[label] = 1
            current_segment = {'data': [], 'label': None}

    else:
        current_segment['data'].append(row)

# Append the last segment if it hasn't been appended yet
if current_segment['data']:
    segments.append(current_segment)

print("\nNumber of Segments Captured:", len(segments))

# Iterate over segments and print each one's details
for segment in segments:
    label = segment['label']
    # Initialize segment count if it's the first time this label is being captured
    if label not in segment_counts:
        segment_counts[label] = 1
    # Convert segment data to DataFrame
    segment_df = pd.DataFrame(segment['data'])
    segment_df = segment_df.loc[:, ~segment_df.columns.duplicated()]
    # Print each segment's information with label-specific numbering
    print(f"\nSegment {segment_counts[label]} label: {label}, Data points: {len(segment['data'])}")
    print(segment_df.head())
    segment_counts[label] += 1  # Increment segment count for the label


Streaming output truncated to the last 5000 lines.
10901   -51.81     7.69   Yes  
10902   -51.77     7.63   Yes  
10903   -51.76     7.68   Yes  
10904   -51.78     7.68   Yes  
10905   -51.75     7.67   Yes  

Segment 44 label: Yes, Data points: 76
       angle1  angle2  angle3  angle4  angle5  rolldeg  pitchdeg  anglegx  \
10979  203.00  105.00     1.0   281.0    45.0    34.63    -33.33    20.65   
10980  204.00  105.00     1.0   288.0    45.0    79.58     -1.00    20.61   
10981  206.00  105.00     1.0   271.0    45.0    69.22    -14.24    20.63   
10982  206.00  106.00     1.0   283.0    45.0    65.46    -18.35    20.64   
10983  206.00  105.00     1.0   285.0    45.0    78.05     11.91    20.65   

       anglegy  anglegz label  
10979   -52.18     8.02   Yes  
10980   -52.15     8.06   Yes  
10981   -52.15     8.06   Yes  
10982   -52.23     8.08   Yes  
10983   -52.25     8.07   Yes  

Segment 45 label: Yes, Data points: 77
       angle1  angle2  angle3  angle4  angle5  rolldeg

In [ ]:
# if 'Start' in str(row['angle1']):
#     if current_segment['data']:  # If current segment has data, save it before starting new
#         segments.append(current_segment)
#         print(f"Appending segment for label {current_segment['label']} with count {segment_counts[current_segment['label']]}")
#     current_segment = {'data': [], 'label': row['label']}
#     if row['label'] not in segment_counts:
#         segment_counts[row['label']] = 1  # Initialize or reset
#         print(f"New label detected: {row['label']}, starting count at {segment_counts[row['label']]}")
#     else:
#         segment_counts[row['label']] += 1

In [ ]:
# Extract unique labels
unique_labels = set(segment['label'] for segment in segments)

# Print unique labels
print("Unique Labels:")
for label in unique_labels:
    print(label)


Unique Labels:
Same
How are you
No
Hello
Yes


In [ ]:
# Convert the first two values in each data point to floats and remove the last column
for segment in segments:
    segment['data'] = [[float(value) if i < 2 else value for i, value in enumerate(data_point[:-1])] for data_point in segment['data']]


In [ ]:
import numpy as np

# Extract unique labels
unique_labels = set(segment['label'] for segment in segments)

# Convert unique labels to a NumPy array
labels = np.array(list(unique_labels))

# Print unique labels array
print("Unique Labels Array:")
print(labels)


Unique Labels Array:
['Same' 'How are you' 'No' 'Hello' 'Yes']


In [ ]:
# Print a sample segment
sample_segment = segments[0]  # Assuming you want to print the first segment
print(f"Segment label: {sample_segment['label']}")
print("Sample data points:")
for data_point in sample_segment['data']:  # Print the first 5 data points as a sample
    print(data_point)

Segment label: Hello
Sample data points:
[189.0, 196.0, 62.0, 138.0, 74.0, 30.78, 47.86, 0.75, -0.92, -1.55]
[190.0, 196.0, 62.0, 138.0, 74.0, 30.58, 54.24, 0.81, -0.91, -2.07]
[190.0, 196.0, 62.0, 138.0, 74.0, 64.61, 22.42, 0.84, -0.93, -2.57]
[189.0, 196.0, 62.0, 138.0, 74.0, 74.68, 14.98, 0.74, -0.92, -3.0]
[190.0, 196.0, 63.0, 138.0, 74.0, 71.53, 18.47, 0.85, -0.92, -3.45]
[189.0, 196.0, 62.0, 138.0, 74.0, 74.7, 14.86, 0.8, -0.97, -3.82]
[190.0, 196.0, 63.0, 138.0, 74.0, 79.98, 9.87, 0.88, -0.99, -4.05]
[189.0, 199.0, 62.0, 138.0, 74.0, 80.52, 9.01, 0.87, -1.02, -4.15]
[190.0, 199.0, 63.0, 138.0, 74.0, 79.22, 10.64, 0.86, -1.02, -4.2]
[190.0, 196.0, 62.0, 138.0, 74.0, 79.04, 10.96, 0.86, -1.02, -4.2]
[190.0, 197.0, 62.0, 138.0, 74.0, 78.37, 11.63, 0.89, -1.01, -4.17]
[189.0, 196.0, 61.0, 138.0, 74.0, 78.64, 11.35, 0.91, -1.01, -4.11]
[190.0, 196.0, 61.0, 138.0, 74.0, 77.24, 12.74, 0.92, -1.01, -4.06]
[190.0, 196.0, 63.0, 138.0, 74.0, 75.94, 14.06, 0.94, -0.99, -4.01]
[189.0, 197.0,

In [ ]:
# Assuming 'segments' is a list of dictionaries where each dictionary has a 'label' key

# Initialize a dictionary to count segments for each label
segments_count = {}

# Count the segments for each label
for segment in segments:
    label = segment['label']
    if label in segments_count:
        segments_count[label] += 1
    else:
        segments_count[label] = 1

# Print the total number of segments for each label
print("Total number of segments for each label:")
for label, count in segments_count.items():
    print(f"Label: {label}, Segments: {count}")


Total number of segments for each label:
Label: Hello, Segments: 99
Label: Yes, Segments: 99
Label: Same, Segments: 99
Label: No, Segments: 99
Label: How are you, Segments: 79


In [ ]:
import numpy as np
import os

# Assuming 'segments' is defined as before
for segment in segments:
    # Create an array for each sample in the segment data
    segment['data'] = [np.array(sample) if not isinstance(sample, np.ndarray) else sample for sample in segment['data']]

# Now proceed with the rest of your code that depends on segment['data'] being an array
DATA_PATH = "dataCollection2"  # Update this with your desired data path

for segment in segments:
    label = segment['label']
    segment_data = segment['data']

    # Create a folder for the label if it doesn't exist
    label_folder = os.path.join(DATA_PATH, label)
    if not os.path.exists(label_folder):
        os.makedirs(label_folder)

    # Find the next segment index for this label
    segment_index = len([name for name in os.listdir(label_folder) if os.path.isdir(os.path.join(label_folder, name))])

    # Create a folder for the new segment
    segment_folder = os.path.join(label_folder, f"segment_{segment_index}")
    if not os.path.exists(segment_folder):
        os.makedirs(segment_folder)

    # Save each sample in the segment as a separate .npy file
    for sample_index, sample_data in enumerate(segment_data):
        np.save(os.path.join(segment_folder, f"sample_{sample_index}.npy"), sample_data)


In [ ]:
# Assuming 'segments' is a list of dictionaries where each dictionary has a 'label' and 'data' key
for segment in segments:
    print(f"Label: {segment['label']}")
    for sample_index, sample in enumerate(segment['data']):
        print(f"Sample {sample_index}: {sample}")


Streaming output truncated to the last 5000 lines.
Sample 12: [153.   241.    -4.   364.    35.    56.61   3.98   3.73 -24.36  48.46]
Sample 13: [156.   239.    -4.   364.    35.    56.37   4.33   3.76 -24.35  48.46]
Sample 14: [155.   241.    -3.   364.    35.    57.18   4.09   3.75 -24.35  48.47]
Sample 15: [154.   241.    -4.   364.    35.    56.43   4.48   3.75 -24.35  48.48]
Sample 16: [156.   241.    -3.   364.    35.    56.04   5.39   3.72 -24.3   48.54]
Sample 17: [161.   232.    -3.   367.    35.    58.03   6.35   3.62 -24.06  48.78]
Sample 18: [175.   200.    -3.   371.    35.    61.2    6.23   3.36 -23.71  49.11]
Sample 19: [187.   135.    -3.   375.    34.    61.48   7.73   3.16 -23.2   49.34]
Sample 20: [193.    82.    -2.   386.    34.    58.24  10.37   2.96 -22.3   49.68]
Sample 21: [203.    71.    -2.   394.    33.    59.24   9.62   2.78 -21.25  50.  ]
Sample 22: [209.    71.    -4.   402.    31.    58.42  29.01   2.29 -20.77  50.26]
Sample 23: [206.    64.    -4.   410

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
actions = {seg['label'] for seg in segments}

# Create a mapping from label to a unique integer
label_map = {label: num for num, label in enumerate(actions)}

label_map

{'Same': 0, 'How are you': 1, 'No': 2, 'Hello': 3, 'Yes': 4}

In [ ]:
import numpy as np
import os

# Define your base data path where segments are stored
DATA_PATH = "dataCollection2"

sequences = []
labels = []

# Assuming you have label_map defined somewhere
num_sequences_to_take = 79
num_samples_per_sequence = 68

for action in actions:
    label_folder = os.path.join(DATA_PATH, action)
    sequence_count = len([name for name in os.listdir(label_folder) if os.path.isdir(os.path.join(label_folder, name))])

    for sequence in range(min(sequence_count, num_sequences_to_take)):  # Limit the number of sequences
        window = []

        sequence_path = os.path.join(label_folder, f"segment_{sequence}")
        frame_count = len([name for name in os.listdir(sequence_path) if name.endswith('.npy')])

        # Sample 70 frames if more frames are available, otherwise sample all available frames
        sampled_frame_indices = np.random.choice(frame_count, size=min(frame_count, num_samples_per_sequence), replace=False)

        for frame_num in sampled_frame_indices:
            file_path = os.path.join(sequence_path, f"sample_{frame_num}.npy")
            res = np.load(file_path)
            window.append(res)

        sequences.append(window)
        labels.append(label_map[action])

# Convert sequences and actions to numpy arrays if needed
sequences = np.array(sequences)
labels = np.array(labels)

# Check the shape of the sequences array
# HERE FOR THE SAKE OF THE EXAMPLE WE HAVE 148 SEGMENTS WITH EACH ONE 70 SAMPLES OF 10 FEATURES
print(sequences.shape)


(395, 68, 10)


In [ ]:
#meaning we have 148 "videos"
labels.shape

(395,)

In [ ]:
X = np.array(sequences)
X.shape

(395, 68, 10)

In [ ]:
#converted labels to categorical
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

In [ ]:
#splitting the date with 5% for testing

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
X_train.shape

(375, 68, 10)

In [ ]:
X_test.shape

(20, 68, 10)

### **TRAIN LSTM MODEL**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# labels = pd.DataFrame(list(labels))
# labels
actions

{'Hello', 'How are you', 'No', 'Same', 'Yes'}

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define your feature scaling factors: Increase the first five features' importance
scaling_factors = np.ones((10,))
scaling_factors[:5] = 10  # Scale up the first five features

In [ ]:
model = Sequential([
    FeatureWeightingLayer(scaling_factors=scaling_factors, input_shape=(68, 10)),
    LSTM(64, return_sequences=True, activation='tanh'),
    LSTM(128, return_sequences=True, activation='tanh'),
    LSTM(64, return_sequences=False, activation='tanh'),
    Dense(64, activation='tanh'),
    Dense(32, activation='tanh'),
    Dense(len(np.unique(labels)), activation='softmax')  # Ensure `labels` is correctly defined
])

In [ ]:
from tensorflow.keras.optimizers import Adam

# Compile the model
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [ ]:
# import pandas as pd

# # Check for null values in X_train
# nulls_in_X_train = np.isnan(X_train).sum()
# print("Null values in X_train:")
# print(nulls_in_X_train)

# # Check for null values in y_train
# nulls_in_y_train = np.isnan(y_train).sum()
# print("\nNull values in y_train:")
# print(nulls_in_y_train)


import numpy as np

# Check for NaN values in X_train
nan_values_in_X_train = np.isnan(X_train).sum()
print("NaN values in X_train:", nan_values_in_X_train)

# Check for infinity values in X_train
inf_values_in_X_train = np.isinf(X_train).sum()
print("Infinity values in X_train:", inf_values_in_X_train)

# Check for NaN values in y_train
nan_values_in_y_train = np.isnan(y_train).sum()
print("NaN values in y_train:", nan_values_in_y_train)

# Check for infinity values in y_train
inf_values_in_y_train = np.isinf(y_train).sum()
print("Infinity values in y_train:", inf_values_in_y_train)


NaN values in X_train: 0
Infinity values in X_train: 0
NaN values in y_train: 0
Infinity values in y_train: 0


In [ ]:
model.fit(X_train, y_train, epochs=20, callbacks=[tb_callback])

Epoch 1/20
12/12 [==============================] - 9s 161ms/step - loss: 1.5884 - categorical_accuracy: 0.2107
Epoch 2/20
12/12 [==============================] - 2s 162ms/step - loss: 1.4182 - categorical_accuracy: 0.7973
Epoch 3/20
12/12 [==============================] - 2s 163ms/step - loss: 1.2288 - categorical_accuracy: 0.9787
Epoch 4/20
12/12 [==============================] - 3s 257ms/step - loss: 0.9930 - categorical_accuracy: 0.9893
Epoch 5/20
12/12 [==============================] - 2s 168ms/step - loss: 0.7541 - categorical_accuracy: 0.9920
Epoch 6/20
12/12 [==============================] - 2s 162ms/step - loss: 0.5616 - categorical_accuracy: 0.9973
Epoch 7/20
12/12 [==============================] - 2s 159ms/step - loss: 0.4156 - categorical_accuracy: 0.9973
Epoch 8/20
12/12 [==============================] - 2s 161ms/step - loss: 0.2939 - categorical_accuracy: 0.9973
Epoch 9/20
12/12 [==============================] - 2s 161ms/step - loss: 0.2062 - categorical_accuracy:

KeyboardInterrupt: 

In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_weighting_layer (F  (None, 68, 10)            10        
 eatureWeightingLayer)                                           
                                                                 
 lstm (LSTM)                 (None, 68, 64)            19200     
                                                                 
 lstm_1 (LSTM)               (None, 68, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                        

In [ ]:
res = model.predict(X_test)


1/1 [==============================] - 1s 1s/step


In [ ]:
# Convert the set 'actions' to a list
actions_list = list(actions)

# Access the element using the index returned by np.argmax(res[0])
predicted_index = np.argmax(res[4])
predicted_action = actions_list[predicted_index]

print("Predicted action:", predicted_action)


Predicted action: How are you


In [ ]:
predicted_index = np.argmax(y_test[4])
predicted_action = actions_list[predicted_index]

print("Predicted label:", predicted_action)

Predicted label: How are you


# **EVALUATION**

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score


In [ ]:
yhat = model.predict(X_test)


1/1 [==============================] - 0s 62ms/step


In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[15,  0],
        [ 0,  5]],

       [[14,  0],
        [ 0,  6]],

       [[18,  0],
        [ 0,  2]],

       [[17,  0],
        [ 0,  3]],

       [[16,  0],
        [ 0,  4]]])

In [ ]:
accuracy_score(ytrue, yhat)

1.0

In [ ]:
# Assuming 'model' is your trained Keras model
model.save('model.h5')  # Saves the entire model to a single HDF5 file


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model

# Assuming FeatureWeightingLayer is defined as shown above
custom_objects = {'FeatureWeightingLayer': FeatureWeightingLayer}

# Load the entire model back
model = load_model('model.h5', custom_objects=custom_objects)


In [ ]:
files.download('model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>